In [1]:
# Importing required libraries
import numpy as np
import pandas as pd

In [2]:
# Loading the dataset
dataset = pd.read_csv('<path>')
dataset.head()

,A,B,C,D
0,5.1,3.5,1.4,2.912
1,4.9,3.0,1.4,2.660
2,4.7,3.2,1.3,2.470
3,4.6,3.1,1.5,2.940
4,5.0,3.6,1.4,2.912


In [3]:
# Seperating variables 
A = dataset['A'].values
B = dataset['B'].values
C = dataset['C'].values
D = dataset['D'].values

In [4]:
# Declaring required variables and creating Y(dependent variables), b(coefficients) and X(independent variables) 
m = len(B)
x0 = np.ones(m)
X = np.array([x0, B, C, D]).T
# Initial Coefficients
b = np.array([0, 0, 0, 0])
Y = np.array(A)
alpha = 0.0001

In [5]:
# Declaring the cost function
def cost_function(X, Y, b):
    m = len(Y)
    J = np.sum((X.dot(b) - Y) ** 2)/(2 * m)
    return J

In [6]:
# Determining the cost for the equation
inital_cost = cost_function(X, Y, b)
print(inital_cost)

12.5909


In [7]:
# Defining the gradient descent model 
def gradient_descent(X, Y, b, alpha, iterations):
    cost_history = [0] * iterations
    m = len(Y)
    
    for iteration in range(iterations):
        # Hypothesis Values
        h = X.dot(b)
        # Difference b/w Hypothesis and Actual Y
        loss = h - Y
        # Gradient Calculation
        gradient = X.T.dot(loss) / m
        # Changing Values of B using Gradient
        b = b - alpha * gradient
        # New Cost Value
        cost = cost_function(X, Y, b)
        cost_history[iteration] = cost
        
    return b, cost_history

In [8]:
# 100000 Iterations
newB, cost_history = gradient_descent(X, Y, b, alpha, 100000)

# New Values of B
print(newB)

# Final Cost of new B
print(cost_history[-1])

[0.3824969  0.91882331 0.24332975 0.37695787]
0.04331599625322742


In [9]:
# Model Evaluation - RMSE
def rmse(Y, Y_pred):
    rmse = np.sqrt(sum((Y - Y_pred) ** 2) / len(Y))
    return rmse

# Model Evaluation - R2 Score
def r2_score(Y, Y_pred):
    mean_y = np.mean(Y)
    ss_tot = sum((Y - mean_y) ** 2)
    ss_res = sum((Y - Y_pred) ** 2)
    r2 = 1 - (ss_res / ss_tot)
    return r2

Y_pred = X.dot(newB)

print("RMSE: ",rmse(Y, Y_pred))
print("R2: ",r2_score(Y, Y_pred))

RMSE:  0.2943331318531008
R2:  0.2885254056498242


In [10]:
# A value of lower RMSE and higher R2 is needed for a model to be fit enough and as it can be seen from the output 
# the value of RMSE and R2 shows the model is not good enough

In [11]:
# Declaring the ols model class using the normalised equation
class normalised_OLS(object):
    
    # Defining initialisation function with decalring the coefficients b
    def __init__(self):
        self.coefficients = []
    
    # Defining the fit function using the normal equation
    def fit(self, X, y):
        if len(X.shape) == 1:
            X = self._reshape_x(X)
        
        X = self._concatenate_ones(X)
        self.coefficients = np.linalg.inv(X.transpose().dot(X)).dot(X.transpose()).dot(y)
        
    # Defining predict function to determine coefficients
    def predict(self, entry):
        b0 = self.coefficients[0]
        other_betas = self.coefficients[1:]
        prediction = b0
        
        for xi, bi in zip(entry, other_betas):
            prediction += (bi * xi)
        return prediction
    
    # Declaring the reshape function for reshaping the x matrix depending on the shape
    def _reshape_x(self,X):
        return X.reshape(-1,1)
    
    # Defining concatenate function to concat ones matrix in the x matrix
    def _concatenate_ones(self,X):
        ones = np.ones(shape=X.shape[0]).reshape(-1,1)
        return np.concatenate((ones,X),1)

In [12]:
# Creating the X and Y variables for the equation
X = dataset.drop('A',axis=1).values
y = dataset['A'].values

In [13]:
# Creating the object from class
model_nols = normalised_OLS()

In [14]:
# Running the fit method from the class object to fit the model using the variables
model_nols.fit(X,y)

In [15]:
# Determing coefficients 
model_nols.coefficients

array([ 2.25736466,  0.67387756,  0.57370692, -0.1368836 ])

In [21]:
cost_function(np.concatenate((np.ones(shape=X.shape[0]).reshape(-1,1),X),1), y, model_nols.coefficients)

0.026082744207358385

In [ ]:
# Quality check
model_nols.predict(X[0])

In [ ]:
# Creating a list of predicted values
y_preds_nols = []
for row in X:
    y_preds_nols.append(model_nols.predict(row))

In [ ]:
# Evaluating the model
print("RMSE: ",rmse(Y, np.ravel(y_preds_nols)))
print("R2: ",r2_score(Y, np.ravel(y_preds_nols)))

In [ ]:
#-------------------------------------------------------------------------
# As the RMSE and R2 values from the normal equation model are better than the Ordinary Least Square model using gradient descent 
# hence the normal equation model fits better and hence is a better model.
# The normal equation tends to minimize the SSE as it finds the minima by equating to zero and find the coefficients whereas in 
# gradient descent it tends to minimize cost by iterating over the gradient multiple times with precision of alpha to find the 
# minima of the slope which needs to be run infinite times and can lead to missing the exactg minima. 
#-------------------------------------------------------------------------